# FragPipe Mutant Peptide Channel Enrichment

For each detected mutant PSM, parses the **Mapped Proteins** column to find which
mutations the peptide is consistent with, looks up which TMT channels carry those
mutations (via per-sample FASTAs), then computes:

> **ratio = mean RI in channels that SHOULD have the mutation /
>            mean RI in channels that should NOT**

A ratio >> 1 confirms the peptide is concentrated in the expected patient channel(s).

In [ ]:
import glob
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# ── CONFIG ────────────────────────────────────────────────────────────────────
PLEX_ID     = "01CPTAC_CCRCC_Proteome_JHU_20171007"
RESULTS_DIR = f"/scratch/leduc.an/AAS_Evo/MS_SEARCH/results/{PLEX_ID}"
REPO_DIR    = "/home/leduc.an/AAS_Evo_project/AAS_Evo"
TMT_MAP     = f"{REPO_DIR}/metadata/PDC_meta/pdc_file_tmt_map.tsv"
GDC_META    = f"{REPO_DIR}/metadata/GDC_meta/gdc_meta_matched.tsv"
FASTA_DIR   = "/scratch/leduc.an/AAS_Evo/FASTA/per_sample"

# psm.tsv lives in {PLEX_ID}_1/ subdirectory (FragPipe experiment folder)
psm_matches = sorted(glob.glob(os.path.join(RESULTS_DIR, "*_1/psm.tsv")))
if not psm_matches:
    raise FileNotFoundError(f"No *_1/psm.tsv found under {RESULTS_DIR}")
PSM_FILE = psm_matches[0]
print(f"PSM file: {PSM_FILE}")

# ── LOAD PSM ──────────────────────────────────────────────────────────────────
psm = pd.read_csv(PSM_FILE, sep="\t", low_memory=False)
print(f"Total PSMs: {len(psm):,}")

# ── DETECT TMT INTENSITY COLUMNS ──────────────────────────────────────────────
CHANNEL_ORDER = ["126","127N","127C","128N","128C","129N","129C","130N","130C","131N","131C"]

def find_ri_cols(df, channel_order):
    found = {}
    for ch in channel_order:
        if ch in df.columns:
            found[ch] = ch; continue
        candidates = [c for c in df.columns if c.startswith("Intensity") and c.endswith(f"_{ch}")]
        if candidates:
            found[ch] = candidates[0]; continue
        candidates = [c for c in df.columns if c.startswith(ch) and "intensity" in c.lower()]
        if candidates:
            found[ch] = candidates[0]
    return found

ri_col_map = find_ri_cols(psm, CHANNEL_ORDER)
print(f"RI columns found ({len(ri_col_map)}): {list(ri_col_map.keys())}")

if not ri_col_map:
    intensity_cols = [c for c in psm.columns if "intensity" in c.lower()]
    print("WARNING — no RI columns matched. Intensity-related columns found:")
    for c in intensity_cols[:20]: print(f"  {c}")

# ── FILTER: mutant PSMs with non-zero TMT signal ──────────────────────────────
mut_mask = psm["Entry Name"].str.endswith("-mut", na=False)
mut_all  = psm[mut_mask].copy()

ri_cols = list(ri_col_map.values())
if ri_cols:
    nonzero_mask = (mut_all[ri_cols].fillna(0) > 0).any(axis=1)
    mut_psm = mut_all[nonzero_mask].copy()
else:
    mut_psm = mut_all.copy()

print(f"\nMutant PSMs (any RI > 0): {len(mut_psm):,}  "
      f"(of {len(mut_all):,} mutant, {len(psm):,} total)")

In [ ]:
# ── BUILD MUTATION → CHANNELS MAP ────────────────────────────────────────────
TMT_CHANNEL_MAP = {
    "tmt_126":"126",  "tmt_127n":"127N", "tmt_127c":"127C",
    "tmt_128n":"128N","tmt_128c":"128C", "tmt_129n":"129N",
    "tmt_129c":"129C","tmt_130n":"130N", "tmt_130c":"130C",
    "tmt_131":"131N", "tmt_131c":"131C",
}

tmt = pd.read_csv(TMT_MAP, sep="\t")
gdc = pd.read_csv(GDC_META, sep="\t")

# Resolve GDC UUID column — may be 'gdc_file_id' or 'file_id' depending on
# which version of the metadata pipeline generated the file.
if "gdc_file_id" in gdc.columns:
    uuid_col = "gdc_file_id"
elif "file_id" in gdc.columns:
    uuid_col = "file_id"
    gdc = gdc.rename(columns={"file_id": "gdc_file_id"})
    uuid_col = "gdc_file_id"
else:
    raise ValueError(f"Cannot find UUID column in GDC metadata. "
                     f"Columns present: {list(gdc.columns)}")
print(f"GDC UUID column: '{uuid_col}'")

plex_tmt = (tmt[tmt["run_metadata_id"] == PLEX_ID]
            [["tmt_channel","case_submitter_id","sample_type"]].drop_duplicates())
plex_tmt = plex_tmt[~plex_tmt["case_submitter_id"].str.lower()
                    .isin(["ref","reference","pooled","pool","nan"])]
plex_tmt["channel"] = plex_tmt["tmt_channel"].map(TMT_CHANNEL_MAP)

# Diagnostic: check sample_type values align between TMT map and GDC metadata
tmt_sample_types = set(plex_tmt["sample_type"].dropna().str.lower().unique())
gdc_sample_types = set(gdc["sample_type"].dropna().str.lower().unique())
overlap = tmt_sample_types & gdc_sample_types
print(f"TMT map sample_type values:  {sorted(tmt_sample_types)}")
print(f"GDC meta sample_type values: {sorted(gdc_sample_types)}")
if not overlap:
    print("WARNING: No sample_type overlap — merge will produce all NaN UUIDs! "
          "Check normalization of sample_type in both files.")
else:
    print(f"sample_type overlap (OK):    {sorted(overlap)}")

plex_meta = plex_tmt.merge(
    gdc[["gdc_file_id","case_submitter_id","sample_type"]],
    on=["case_submitter_id","sample_type"], how="left")

n_matched = plex_meta["gdc_file_id"].notna().sum()
n_total   = len(plex_meta)
print(f"\nTMT channels in plex:        {n_total}")
print(f"  matched to GDC UUID:       {n_matched}")
print(f"  no GDC match (NaN UUID):   {n_total - n_matched}")

all_patient_channels = set(plex_meta["channel"].dropna().tolist())
print(f"All patient channels ({len(all_patient_channels)}): {sorted(all_patient_channels)}")

mutation_to_channels = defaultdict(set)
missing_fastas = []
# Only channels where a per-sample FASTA was found are included in the ratio
# universe. Channels with no GDC match or missing FASTA are excluded from both
# the numerator and denominator so they don't dilute the ratio computation.
channels_with_fastas = set()

for _, row in plex_meta.iterrows():
    uuid, channel = row["gdc_file_id"], row["channel"]
    if pd.isna(uuid) or pd.isna(channel):
        continue
    fasta_path = os.path.join(FASTA_DIR, f"{uuid}_mutant.fasta")
    if not os.path.exists(fasta_path):
        missing_fastas.append(uuid)
        continue
    channels_with_fastas.add(channel)
    with open(fasta_path) as f:
        for line in f:
            if not line.startswith(">"):
                continue
            parts = line[1:].strip().split("|")
            if len(parts) >= 4 and parts[0] == "mut":
                mutation_to_channels[(parts[1], parts[3])].add(channel)

print(f"Channels with per-sample FASTA ({len(channels_with_fastas)}): {sorted(channels_with_fastas)}")
n_excluded = len(all_patient_channels - channels_with_fastas)
if n_excluded:
    print(f"  Excluded from ratio universe (no FASTA): "
          f"{sorted(all_patient_channels - channels_with_fastas)}")
print(f"Unique (accession, swap) mutations mapped: {len(mutation_to_channels):,}")
print(f"Missing per-sample FASTAs:                 {len(missing_fastas)}")

In [ ]:
# ── LOAD VAF PER (accession, swap, channel) FROM ALL-MISSENSE TABLE ──────────
# Uses the pre-consolidated VEP output. Filter to this plex's UUIDs — no
# per-file globbing needed.

MISSENSE  = "/scratch/leduc.an/AAS_Evo/VEP/all_missense_mutations.tsv"
REF_FASTA = "/scratch/leduc.an/AAS_Evo/SEQ_FILES/uniprot_human_canonical.fasta"

AA3TO1 = {
    "Ala":"A","Arg":"R","Asn":"N","Asp":"D","Cys":"C","Gln":"Q","Glu":"E",
    "Gly":"G","His":"H","Ile":"I","Leu":"L","Lys":"K","Met":"M","Phe":"F",
    "Pro":"P","Ser":"S","Thr":"T","Trp":"W","Tyr":"Y","Val":"V",
}

def parse_hgvsp_to_swap(hgvsp):
    """'ENSP000.4:p.Leu888Pro' or 'p.Leu888Pro' → 'L888P'.
    Uses re.search so the transcript prefix is ignored."""
    m = re.search(r'p\.([A-Z][a-z]{2})(\d+)([A-Z][a-z]{2})', str(hgvsp))
    if m:
        ref = AA3TO1.get(m.group(1))
        alt = AA3TO1.get(m.group(3))
        if ref and alt:
            return f"{ref}{m.group(2)}{alt}"
    return None

# gene symbol → UniProt accession (from GN= field in reference FASTA)
gene_to_acc = {}
with open(REF_FASTA) as f:
    for line in f:
        if line.startswith(">"):
            m = re.search(r'GN=(\S+)', line)
            if m:
                gene_to_acc[m.group(1)] = line.split("|")[1]
print(f"Gene → accession entries: {len(gene_to_acc):,}")

# UUID → channel for this plex
uuid_to_channel = plex_meta.dropna(subset=["gdc_file_id","channel"]) \
                            .set_index("gdc_file_id")["channel"].to_dict()

# Load missense table, filter to this plex's UUIDs
missense = pd.read_csv(MISSENSE, sep="\t", low_memory=False)
plex_missense = missense[missense["sample_id"].isin(uuid_to_channel)]
print(f"Missense rows for this plex: {len(plex_missense):,}")

# Build (accession, swap, channel) → VAF
mutation_channel_vaf = {}
for _, vrow in plex_missense.iterrows():
    vaf     = vrow.get("VAF", np.nan)
    channel = uuid_to_channel.get(vrow["sample_id"])
    if pd.isna(vaf) or not channel:
        continue
    acc  = gene_to_acc.get(str(vrow.get("SYMBOL", "")))
    swap = parse_hgvsp_to_swap(str(vrow.get("HGVSp", "")))
    if acc and swap:
        mutation_channel_vaf[(acc, swap, channel)] = float(vaf)

print(f"(accession, swap, channel) VAF entries: {len(mutation_channel_vaf):,}")

In [ ]:
# ── COMPUTE CHANNEL ENRICHMENT RATIO PER MUTANT PSM ─────────────────────────
# Computes two ratios per PSM:
#   ratio     : mean RI(have channels) / mean RI(not-have channels)
#   ratio_vaf : VAF-weighted mean RI(have channels) / mean RI(not-have channels)
#
# Universe = channels_with_fastas: only channels where we have genomic data
# (per-sample FASTA). Channels with no GDC match or missing FASTA are excluded
# from both numerator and denominator so they don't dilute the ratio.
# Reference/pooled channels are already excluded (filtered in cell 2).

if "mutation_channel_vaf" not in dir():
    print("NOTE: VAF cell not run — ratio_vaf will be NaN for all PSMs.")
    mutation_channel_vaf = {}

print(f"Ratio universe — channels with FASTA ({len(channels_with_fastas)}): "
      f"{sorted(channels_with_fastas)}")

# ── DETECT PRECURSOR INTENSITY COLUMN ────────────────────────────────────────
PRECURSOR_CANDIDATES = ["Intensity", "Precursor Intensity", "MS1 Intensity",
                        "PrecursorIntensity", "precursor_intensity"]
precursor_col = next((c for c in PRECURSOR_CANDIDATES if c in mut_psm.columns), None)
if precursor_col:
    nonzero_prec = (mut_psm[precursor_col].fillna(0) > 0).sum()
    print(f"Precursor intensity: '{precursor_col}'  ({nonzero_prec:,} / {len(mut_psm):,} non-zero)")
else:
    print(f"WARNING: no precursor intensity column found. Checked: {PRECURSOR_CANDIDATES}")
    print(f"Available columns snippet: {list(mut_psm.columns[:30])}")

def parse_mapped_proteins(mapped_str):
    """'sp|P01889-S35A-B4B8|HLA-B-mut, ...' → {('P01889','S35A'), ...}"""
    if pd.isna(mapped_str):
        return set()
    pairs = set()
    for entry in str(mapped_str).split(","):
        parts = entry.strip().split("|")
        if len(parts) >= 2:
            pid_parts = parts[1].split("-")
            if len(pid_parts) >= 3:
                pairs.add((pid_parts[0], pid_parts[1]))
    return pairs

def parse_protein_id(protein_id):
    """'P11047-L888P-F262' → {('P11047','L888P')}.
    Fallback for PSMs where Mapped Proteins is empty or reference-only."""
    if pd.isna(protein_id):
        return set()
    parts = str(protein_id).split("-")
    if len(parts) >= 3:
        return {(parts[0], parts[1])}
    return set()

results = []
n_no_mutations = n_no_channel_info = n_cant_split = 0
n_protein_id_fallback = 0

for _, row in mut_psm.iterrows():
    # Primary source: Mapped Proteins column
    mutations = parse_mapped_proteins(row.get("Mapped Proteins", float("nan")))
    # Fallback: parse the Protein ID of the primary match
    if not mutations:
        mutations = parse_protein_id(row.get("Protein ID", float("nan")))
        if mutations:
            n_protein_id_fallback += 1
    if not mutations:
        n_no_mutations += 1; continue

    have_channels = set()
    for mut_key in mutations:
        have_channels |= mutation_to_channels.get(mut_key, set())
    # Restrict to channels where we have FASTA data (the ratio universe)
    have_channels &= channels_with_fastas
    not_have_channels = channels_with_fastas - have_channels

    if not have_channels or not not_have_channels:
        n_cant_split += 1; continue

    have_ri = [(ch, row[ri_col_map[ch]]) for ch in have_channels
               if ch in ri_col_map and pd.notna(row[ri_col_map[ch]])]
    not_ri  = [row[ri_col_map[ch]] for ch in not_have_channels
               if ch in ri_col_map and pd.notna(row[ri_col_map[ch]])]

    if not have_ri or not not_ri:
        n_no_channel_info += 1; continue

    mean_not  = np.mean(not_ri)
    mean_have = np.mean([r for _, r in have_ri])
    ratio     = mean_have / mean_not if mean_not > 0 else np.nan

    # VAF-weighted: weight each have-channel's RI by its mutation VAF
    weighted_pairs = []
    for ch, ri_val in have_ri:
        vaf = None
        for acc, sw in mutations:
            v = mutation_channel_vaf.get((acc, sw, ch))
            if v is not None:
                vaf = v; break
        if vaf is not None:
            weighted_pairs.append((ri_val, vaf))

    if weighted_pairs:
        ris_v, wts_v = zip(*weighted_pairs)
        ratio_vaf = np.average(ris_v, weights=wts_v) / mean_not if mean_not > 0 else np.nan
    else:
        ratio_vaf = np.nan

    prec_intens = float(row[precursor_col]) \
        if (precursor_col and pd.notna(row.get(precursor_col))
            and row.get(precursor_col, 0) > 0) else np.nan

    results.append({
        "Peptide":            row.get("Peptide", ""),
        "n_have_ch":          len(have_channels),
        "n_not_have_ch":      len(not_have_channels),
        "mean_have_ri":       mean_have,
        "mean_not_ri":        mean_not,
        "ratio":              ratio,
        "ratio_vaf":          ratio_vaf,
        "precursor_intensity": prec_intens,
    })

ratios = pd.DataFrame(results).dropna(subset=["ratio"])
print(f"\nPSMs with computable ratio:         {len(ratios):,}  (of {len(mut_psm):,} mutant PSMs)")
print(f"  via Mapped Proteins:              {len(ratios) - n_protein_id_fallback:,}")
print(f"  via Protein ID fallback:          {n_protein_id_fallback:,}")
print(f"  of which with VAF-weighted ratio: {ratios['ratio_vaf'].notna().sum():,}")
print(f"  with precursor intensity:         {ratios['precursor_intensity'].notna().sum():,}")
print(f"  Skipped — no mutation key:        {n_no_mutations}")
print(f"  Skipped — no channel split:       {n_cant_split}")
print(f"  Skipped — RI data missing:        {n_no_channel_info}")
print(f"\nUnweighted ratio summary:")
print(ratios["ratio"].describe().round(2))

In [ ]:
# ── PLOT: enrichment ratio histograms ────────────────────────────────────────
# Panel 1: Unweighted (all PSMs)
# Panel 2: VAF-weighted (all PSMs)
# Panel 3: Unweighted, channel-specific only (n_have_ch == 1)

r_unw  = ratios["ratio"].replace(0, np.nan).dropna()
r_vaf  = ratios["ratio_vaf"].replace(0, np.nan).dropna()
r_spec = ratios.loc[ratios["n_have_ch"] == 1, "ratio"].replace(0, np.nan).dropna()

all_vals  = pd.concat([r_unw, r_vaf]) if len(r_vaf) else r_unw
bins      = np.logspace(np.log10(all_vals.min()), np.log10(all_vals.max()), 60)
bins_spec = np.logspace(np.log10(r_spec.min()), np.log10(r_spec.max()), 60) \
            if len(r_spec) else bins

med_unw  = r_unw.median()
med_vaf  = r_vaf.median()  if len(r_vaf)  else None
med_spec = r_spec.median() if len(r_spec) else None

fig, axes = plt.subplots(1, 3, figsize=(18, 4), sharey=False)

panels = [
    (axes[0], r_unw,  med_unw,  bins,      "All PSMs — Unweighted",                    "#4878d0"),
    (axes[1], r_vaf,  med_vaf,  bins,      "All PSMs — VAF-weighted",                  "#6acc65"),
    (axes[2], r_spec, med_spec, bins_spec, "Channel-specific PSMs (n_have=1)\nUnweighted", "#e07b39"),
]

for ax, data, med, b, title, color in panels:
    if len(data) == 0:
        ax.text(0.5, 0.5, "No data", ha="center", va="center", transform=ax.transAxes)
    else:
        ax.hist(data, bins=b, color=color, edgecolor="white", linewidth=0.4)
        ax.axvline(x=1,   color="grey",    linestyle="--", linewidth=1.2, label="ratio = 1")
        ax.axvline(x=med, color="#e74c3c", linestyle="-",  linewidth=1.5,
                   label=f"median = {med:.2f}")
        ax.set_xscale("log")
        ax.legend(fontsize=8)
    ax.set_xlabel("mean RI (have) / mean RI (not-have)  [log scale]")
    ax.set_ylabel("Number of mutant PSMs")
    ax.set_title(f"{title}\n(n={len(data):,})")

fig.suptitle(f"Per-PSM channel enrichment — {PLEX_ID}", fontsize=11, y=1.02)
plt.tight_layout()
fig.savefig(os.path.join(RESULTS_DIR, "mutant_channel_enrichment.pdf"), bbox_inches="tight")
plt.show()

print(f"{'Metric':<32} {'Unweighted':>11} {'VAF-weighted':>13} {'n_have=1':>10}")
print("-" * 68)
for label, d1, d2, d3 in [
    ("Median ratio",
     f"{med_unw:.2f}", f"{med_vaf:.2f}" if med_vaf else "—", f"{med_spec:.2f}" if med_spec else "—"),
    ("% ratio > 1",
     f"{100*(r_unw>1).mean():.1f}%", f"{100*(r_vaf>1).mean():.1f}%" if len(r_vaf) else "—",
     f"{100*(r_spec>1).mean():.1f}%" if len(r_spec) else "—"),
    ("% ratio > 2",
     f"{100*(r_unw>2).mean():.1f}%", f"{100*(r_vaf>2).mean():.1f}%" if len(r_vaf) else "—",
     f"{100*(r_spec>2).mean():.1f}%" if len(r_spec) else "—"),
    ("% ratio > 5",
     f"{100*(r_unw>5).mean():.1f}%", f"{100*(r_vaf>5).mean():.1f}%" if len(r_vaf) else "—",
     f"{100*(r_spec>5).mean():.1f}%" if len(r_spec) else "—"),
    ("N PSMs", f"{len(r_unw):,}", f"{len(r_vaf):,}", f"{len(r_spec):,}"),
]:
    print(f"{label:<32} {d1:>11} {d2:>13} {d3:>10}")

In [ ]:
# ── PER-MUTATION SUMMARY TABLE ─────────────────────────────────────────────────
# One row per mutation that has at least one PSM. Columns:
#   plex_vaf          : mean DNA VAF across channels carrying the mutation
#   ref_peptide       : reference tryptic sequence at same site (swap reversed)
#   ref_detected      : whether ref_peptide appeared in any PSM
#   other_muts_at_site: other mutations sharing the same tryptic region with PSMs
#   mut_site_fraction : mut intensity / (mut + ref + other_mut) — MS-level allele fraction
#   ref_site_fraction : ref intensity / (mut + ref + other_mut)
#   mean_prec_intensity: mean MS1 precursor intensity across mut PSMs
#   PEP / Expectation : best identifiability score across mut PSMs (lower = better)

# ── Score column (identifiability) ───────────────────────────────────────────
SCORE_COLS = [
    ("PeptideProphet Probability", "PEP",         False),  # convert: PEP = 1 - prob
    ("PepProphet Prob",            "PEP",         False),
    ("Expectation",                "Expectation", True),   # already lower = better
    ("Hyperscore",                 "Hyperscore",  False),  # higher = better; inverted below
]
score_col_sum, score_label, score_lower_better = None, "N/A", True
for _sc, _sl, _lb in SCORE_COLS:
    if _sc in psm.columns:
        score_col_sum, score_label, score_lower_better = _sc, _sl, _lb
        break
is_prob_col = score_col_sum and "Probability" in score_col_sum
print(f"Identifiability column: '{score_col_sum}'  →  reported as '{score_label}' (lower = better)")

# ── Peptide-level lookups from the FULL PSM table (all proteins) ──────────────
pep_prec_intens = defaultdict(list)   # pep.upper() → [precursor intensities]
pep_scores_all  = defaultdict(list)   # pep.upper() → [score values]

for _, row in psm.iterrows():
    pep = str(row.get("Peptide", "")).upper().strip()
    if not pep: continue
    pi = row.get(precursor_col, np.nan) if precursor_col else np.nan
    sc = row.get(score_col_sum, np.nan) if score_col_sum else np.nan
    if not pd.isna(pi) and float(pi) > 0:
        pep_prec_intens[pep].append(float(pi))
    if not pd.isna(sc):
        pep_scores_all[pep].append(float(sc))

all_detected_peps = set(str(p).upper().strip() for p in psm["Peptide"].dropna())
print(f"Unique detected peptides: {len(all_detected_peps):,}")

# ── Assign primary mutation key to each mut_psm row ──────────────────────────
def get_primary_mutation(row):
    muts = parse_mapped_proteins(row.get("Mapped Proteins", ""))
    if not muts:
        muts = parse_protein_id(row.get("Protein ID", ""))
    return next(iter(muts), None)

mut_key_to_psm_rows = defaultdict(list)
for _, row in mut_psm.iterrows():
    key = get_primary_mutation(row)
    if key:
        mut_key_to_psm_rows[key].append(row)

# ── Derive reference peptide from mutant peptide + swap code ─────────────────
def derive_ref_candidates(mut_pep, swap):
    """'VVVDGGGK', 'A117D' → ['VVVAGGGK'].
    Replaces each occurrence of mut_aa in mut_pep with ref_aa.
    Returns list of candidate reference peptides (usually just one)."""
    m = re.match(r'^([A-Z])(\d+)([A-Z])$', str(swap))
    if not m: return []
    ref_aa, mut_aa = m.group(1), m.group(3)
    pep = str(mut_pep).upper()
    return [pep[:i] + ref_aa + pep[i+1:] for i, aa in enumerate(pep) if aa == mut_aa]

# First pass: resolve mut_pep, ref_pep, ref_detected for each (accession, swap)
mut_info = {}             # (accession, swap) → dict with mut_pep, ref_pep, ref_detected
ref_pep_to_mut_keys = defaultdict(set)   # ref_pep → {(acc, sw), ...}

for (accession, swap), rows_list in mut_key_to_psm_rows.items():
    # Most common peptide sequence wins (handles rare mis-assignments)
    pep_counts = defaultdict(int)
    for r in rows_list:
        pep_counts[str(r.get("Peptide", "")).upper().strip()] += 1
    mut_pep = max(pep_counts, key=pep_counts.get) if pep_counts else None
    if not mut_pep:
        continue

    ref_cands = [rc.upper() for rc in derive_ref_candidates(mut_pep, swap)]
    # Prefer a candidate that actually appears in the PSMs (detected ref)
    ref_pep_det = next((rc for rc in ref_cands if rc in all_detected_peps), None)
    ref_pep_use = ref_pep_det or (ref_cands[0] if ref_cands else None)

    mut_info[(accession, swap)] = {
        "mut_pep":      mut_pep,
        "ref_pep":      ref_pep_use,
        "ref_detected": (ref_pep_det is not None) if ref_cands else None,
    }
    if ref_pep_use:
        ref_pep_to_mut_keys[ref_pep_use].add((accession, swap))

# ── Build summary table ────────────────────────────────────────────────────────
summary_rows = []
for (accession, swap), channels in mutation_to_channels.items():
    info = mut_info.get((accession, swap))
    if not info:
        continue   # mutation in per-sample FASTA but no PSMs

    mut_pep = info["mut_pep"]
    ref_pep = info["ref_pep"]
    ref_det = info["ref_detected"]

    # Gene from Entry Name (format: GENE-mut)
    rows_list = mut_key_to_psm_rows[(accession, swap)]
    entry_name = str(rows_list[0].get("Entry Name", "")) if rows_list else ""
    gene = entry_name[:-4] if entry_name.endswith("-mut") else entry_name

    # Plex VAF: mean DNA VAF across channels for this mutation
    vafs = [mutation_channel_vaf[(accession, swap, ch)]
            for ch in channels if (accession, swap, ch) in mutation_channel_vaf]
    plex_vaf = np.mean(vafs) if vafs else np.nan

    # Other detected mutations sharing the same tryptic site (same ref_pep)
    other_muts = set()
    if ref_pep:
        for (other_acc, other_sw) in ref_pep_to_mut_keys.get(ref_pep, set()):
            if (other_acc, other_sw) != (accession, swap):
                other_muts.add(f"{other_acc}:{other_sw}")

    # Site-level precursor intensities (sum across all PSMs of each variant)
    mut_intens   = pep_prec_intens.get(mut_pep, [])
    ref_intens   = pep_prec_intens.get(ref_pep, []) if ref_pep else []
    other_intens = []
    for om_key in other_muts:
        om_acc, om_sw = om_key.split(":", 1)
        om_inf = mut_info.get((om_acc, om_sw))
        if om_inf:
            other_intens += pep_prec_intens.get(om_inf["mut_pep"], [])

    total_site = sum(mut_intens) + sum(ref_intens) + sum(other_intens)
    mut_frac = sum(mut_intens) / total_site if total_site > 0 else np.nan
    ref_frac = sum(ref_intens) / total_site if total_site > 0 else np.nan

    # Best identifiability score across PSMs of this peptide
    scores = pep_scores_all.get(mut_pep, [])
    if scores:
        if is_prob_col:
            best_sc = 1 - max(scores)        # PEP = 1 - PeptideProphet Prob
        elif score_lower_better:
            best_sc = min(scores)
        else:
            best_sc = max(scores)
    else:
        best_sc = np.nan

    summary_rows.append({
        "accession":           accession,
        "gene":                gene,
        "swap":                swap,
        "n_channels":          len(channels),
        "plex_vaf":            plex_vaf,
        "n_psms":              len(rows_list),
        "mut_peptide":         mut_pep,
        "ref_peptide":         ref_pep,
        "ref_detected":        ref_det,
        "other_muts_at_site":  "; ".join(sorted(other_muts)) or "",
        "mut_site_fraction":   mut_frac,
        "ref_site_fraction":   ref_frac,
        "mean_prec_intensity": np.mean(mut_intens) if mut_intens else np.nan,
        score_label:           best_sc,
    })

summary = pd.DataFrame(summary_rows)
print(f"\nPer-mutation summary: {len(summary):,} mutations with PSMs")
print(f"  ref_detected = True:           {summary['ref_detected'].eq(True).sum():,}")
print(f"  ref_detected = False:          {summary['ref_detected'].eq(False).sum():,}")
print(f"  ref_detected = None:           {summary['ref_detected'].isna().sum():,}")
print(f"  with other muts at same site:  {(summary['other_muts_at_site'] != '').sum():,}")
print(f"\nMS-level allele fraction (mut_site_fraction):")
print(summary["mut_site_fraction"].describe().round(3))
print(f"\nDNA VAF (plex mean):")
print(summary["plex_vaf"].describe().round(3))

display(summary.sort_values("mut_site_fraction", ascending=False).head(20))

In [ ]:
# ── DETECTION RATE BY VAF BIN ─────────────────────────────────────────────────
# Loops over ALL mutations in mutation_to_channels (everything in the per-plex
# FASTA, detected or not) and asks two questions per VAF bin:
#   1. Was a mutant PSM detected?  (mut_detected)
#   2. Among detected, was the reference tryptic peptide also found?  (ref_detected)
#
# ref_detected = True means the site was sampled by MS and the protein is
# expressed — a mut PSM at such a site is more trustworthy than one where
# even the reference peptide is absent.

detected_keys = set(mut_key_to_psm_rows.keys())   # (acc, sw) with ≥1 PSM
ref_det_lookup = {k: v["ref_detected"] for k, v in mut_info.items()}

all_mut_rows = []
for (accession, swap), channels in mutation_to_channels.items():
    vafs = [mutation_channel_vaf[(accession, swap, ch)]
            for ch in channels if (accession, swap, ch) in mutation_channel_vaf]
    plex_vaf    = np.mean(vafs) if vafs else np.nan
    mut_det     = (accession, swap) in detected_keys
    ref_det     = ref_det_lookup.get((accession, swap))   # None if no PSM
    all_mut_rows.append({
        "accession":    accession,
        "swap":         swap,
        "n_channels":   len(channels),
        "plex_vaf":     plex_vaf,
        "mut_detected": mut_det,
        "ref_detected": ref_det,
    })

all_mut_df = pd.DataFrame(all_mut_rows)
has_vaf = all_mut_df.dropna(subset=["plex_vaf"]).copy()

VAF_EDGES  = [0, 0.10, 0.20, 0.30, 0.50, 1.001]
VAF_LABELS = ["0.00–0.10", "0.10–0.20", "0.20–0.30", "0.30–0.50", "0.50–1.00"]
has_vaf["vaf_bin"] = pd.cut(has_vaf["plex_vaf"], bins=VAF_EDGES, labels=VAF_LABELS)

print(f"All mutations in plex FASTA:  {len(all_mut_df):,}")
print(f"  with VAF data:              {len(has_vaf):,}")
print(f"  without VAF data:           {all_mut_df['plex_vaf'].isna().sum():,}")
print()

hdr = f"  {'VAF bin':<12} {'Total':>7}  {'Mut det':>8}  {'Det%':>6}  {'Ref det|mut':>12}  {'Ref%|mut':>9}"
print(hdr)
print("  " + "-" * 60)

# Totals row
def _row(label, grp):
    n_tot  = len(grp)
    n_mdet = grp["mut_detected"].sum()
    det_pct = f"{100*n_mdet/n_tot:.1f}%" if n_tot else "—"
    det_grp = grp[grp["mut_detected"]]
    n_rdet  = det_grp["ref_detected"].eq(True).sum()
    ref_pct = f"{100*n_rdet/len(det_grp):.1f}%" if len(det_grp) else "—"
    print(f"  {label:<12} {n_tot:>7,}  {n_mdet:>8,}  {det_pct:>6}  {n_rdet:>12,}  {ref_pct:>9}")

for b in VAF_LABELS:
    _row(b, has_vaf[has_vaf["vaf_bin"] == b])

print("  " + "-" * 60)
_row("ALL w/ VAF", has_vaf)
_row("ALL",        all_mut_df)

print()
print("Columns:")
print("  Mut det     : mutations with ≥1 PSM identified")
print("  Det%        : detection rate (mut PSM found / total in FASTA)")
print("  Ref det|mut : among detected, reference peptide also found in PSMs")
print("  Ref%|mut    : fraction of detected mutations with reference site covered")

In [ ]:
# ── DETECTION RATE CONTEXT: mutant vs reference ────────────────────────────────
# The 6.5% mutant detection rate needs context. The key asymmetry is:
#   - Each mutant FASTA entry  = exactly 1 tryptic peptide (the one containing
#                                the mutation), generated with ≤1 missed cleavage
#   - Each reference FASTA entry = full protein with many tryptic peptides
#
# We compute the ACTUAL theoretical tryptic peptide count from the reference
# FASTA using the same cleavage rules as the mutant peptide generation:
#   - Cleave C-terminal to K/R, not before P
#   - ≤1 missed cleavage
#   - Length 7–50 aa (standard LC-MS/MS detectable range)

def trypsin_digest(seq, missed_cleavages=1, min_len=7, max_len=50):
    """In-silico trypsin digest: cleave after K/R not followed by P."""
    cuts = [0]
    for i in range(len(seq) - 1):
        if seq[i] in ('K', 'R') and seq[i + 1] != 'P':
            cuts.append(i + 1)
    cuts.append(len(seq))
    peptides = set()
    n = len(cuts) - 1
    for i in range(n):
        for mc in range(missed_cleavages + 1):
            if i + mc + 1 <= n:
                pep = seq[cuts[i]:cuts[i + mc + 1]]
                if min_len <= len(pep) <= max_len:
                    peptides.add(pep)
    return peptides

# Load reference protein sequences from FASTA (REF_FASTA defined in cell 3)
print("Computing in-silico tryptic digest of reference proteome...")
ref_seqs = {}   # accession → sequence
current_acc, current_seq = None, []
with open(REF_FASTA) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith(">"):
            if current_acc:
                ref_seqs[current_acc] = "".join(current_seq)
            parts = line.split("|")
            current_acc = parts[1] if len(parts) >= 2 else line[1:].split()[0]
            current_seq = []
        else:
            current_seq.append(line)
    if current_acc:
        ref_seqs[current_acc] = "".join(current_seq)

N_REF_PROTEINS = len(ref_seqs)
print(f"  Reference proteins loaded: {N_REF_PROTEINS:,}")

# Digest all reference proteins — count both unique peptides and per-protein avg
all_theoretical_peps = set()
pep_counts_per_prot  = []
for seq in ref_seqs.values():
    peps = trypsin_digest(seq, missed_cleavages=1, min_len=7, max_len=50)
    all_theoretical_peps.update(peps)
    pep_counts_per_prot.append(len(peps))

n_theoretical_ref_peps = len(all_theoretical_peps)
avg_pep_per_prot       = np.mean(pep_counts_per_prot)
med_pep_per_prot       = np.median(pep_counts_per_prot)
print(f"  Unique theoretical tryptic peptides (≤1 MC, 7–50 aa): {n_theoretical_ref_peps:,}")
print(f"  Per-protein: mean={avg_pep_per_prot:.0f}, median={med_pep_per_prot:.0f}")

# ── Reference protein coverage from psm.tsv ──────────────────────────────────
ref_mask      = ~psm["Entry Name"].str.endswith(("-mut", "-comp"), na=True)
mut_comp_mask =  psm["Entry Name"].str.endswith(("-mut", "-comp"), na=False)

ref_proteins_detected = psm.loc[ref_mask, "Entry Name"].dropna().nunique()
ref_peps_detected     = psm.loc[ref_mask, "Peptide"].dropna().nunique()
mut_peps_detected     = psm.loc[mut_comp_mask, "Peptide"].dropna().nunique()
ref_psms              = ref_mask.sum()
mut_psms              = mut_comp_mask.sum()
N_MUT_ENTRIES         = len(mutation_to_channels)

print()
print("=" * 62)
print("DETECTION RATE COMPARISON")
print("=" * 62)
print(f"{'Metric':<44} {'Reference':>9} {'Mutant':>8}")
print("-" * 62)
print(f"{'FASTA entries (proteins / mutant peptides)':<44} {N_REF_PROTEINS:>9,} {N_MUT_ENTRIES:>8,}")
print(f"{'Theoretical tryptic peptides (≤1 MC, 7–50 aa)':<44} {n_theoretical_ref_peps:>9,} {N_MUT_ENTRIES:>8,}")
print(f"{'Detected unique peptide sequences':<44} {ref_peps_detected:>9,} {mut_peps_detected:>8,}")
print(f"{'PSMs':<44} {ref_psms:>9,} {mut_psms:>8,}")
print()
print(f"  Protein coverage:  {ref_proteins_detected:,} / {N_REF_PROTEINS:,} proteins "
      f"= {100 * ref_proteins_detected / N_REF_PROTEINS:.1f}%")
print(f"  Per-tryptic-peptide detection rate:")
print(f"    Reference: {ref_peps_detected:,} detected / {n_theoretical_ref_peps:,} theoretical "
      f"= {100 * ref_peps_detected / n_theoretical_ref_peps:.2f}%")
print(f"    Mutant:    {mut_peps_detected:,} detected / {N_MUT_ENTRIES:,} in FASTA   "
      f"= {100 * mut_peps_detected / N_MUT_ENTRIES:.2f}%")
print()
print("NOTE: theoretical reference count includes non-unique peptides shared")
print("across proteins; per-protein avg is the more intuitive number.")
print(f"  Avg tryptic peptides per reference protein: {avg_pep_per_prot:.0f} "
      f"(median: {med_pep_per_prot:.0f})")


In [ ]:
# ── PLOT: ratio by precursor intensity quintile (box plots) ──────────────────
# Tests hypothesis 1: RI background noise drives the diluted ratios.
# PSMs are split into 5 equal-N bins by log10(precursor intensity).
# If background noise is the problem, low-intensity PSMs (weaker signal,
# more noise-dominated RI) should show the worst ratios; high-intensity PSMs
# should show clearly better enrichment.

r_prec = ratios.dropna(subset=["precursor_intensity"]).copy()
r_prec = r_prec[r_prec["precursor_intensity"] > 0]

if len(r_prec) < 10:
    print(f"Insufficient data: only {len(r_prec)} PSMs have precursor intensity > 0.")
    if not precursor_col:
        print("No precursor intensity column was found in psm.tsv — check column names.")
else:
    log_intens = np.log10(r_prec["precursor_intensity"])
    try:
        r_prec["intens_bin"] = pd.qcut(log_intens, q=5, duplicates="drop")
    except ValueError:
        r_prec["intens_bin"] = pd.qcut(log_intens, q=4, duplicates="drop")

    bin_order = list(r_prec["intens_bin"].cat.categories)

    panel_specs = [
        ("ratio",     "All PSMs — Unweighted",                        "#4878d0", r_prec),
        ("ratio_vaf", "All PSMs — VAF-weighted",                      "#6acc65",
         r_prec[r_prec["ratio_vaf"].notna()]),
        ("ratio",     "Channel-specific (n_have_ch = 1)\nUnweighted", "#e07b39",
         r_prec[r_prec["n_have_ch"] == 1]),
    ]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=False)

    for ax, (col, title, color, df) in zip(axes, panel_specs):
        data_by_bin = []
        tick_labels = []
        for b in bin_order:
            grp = df[df["intens_bin"] == b][col].replace(0, np.nan).dropna()
            data_by_bin.append(grp.values if len(grp) else np.array([np.nan]))
            tick_labels.append(f"10^{b.left:.1f}–\n10^{b.right:.1f}\n(n={len(grp):,})")

        ax.boxplot(
            data_by_bin,
            patch_artist=True,
            showfliers=True,
            flierprops=dict(marker=".", markersize=2, alpha=0.25, color=color),
            medianprops=dict(color="#e74c3c", linewidth=2),
            boxprops=dict(facecolor=color, alpha=0.45),
            whiskerprops=dict(color="grey", linewidth=1),
            capprops=dict(color="grey", linewidth=1),
        )
        ax.axhline(y=1, color="grey", linestyle="--", linewidth=1.2, label="ratio = 1")
        ax.set_yscale("log")
        ax.set_xticks(range(1, len(bin_order) + 1))
        ax.set_xticklabels(tick_labels, fontsize=7.5)
        ax.set_xlabel(f"Precursor intensity ('{precursor_col}') — low → high", fontsize=9)
        ax.set_ylabel("mean RI (have) / mean RI (not-have)  [log scale]")
        total_n = df[col].replace(0, np.nan).dropna().__len__()
        ax.set_title(f"{title}\n(n = {total_n:,} with intensity data)")
        ax.legend(fontsize=8)

    fig.suptitle(
        f"Channel enrichment vs precursor intensity — {PLEX_ID}\n"
        f"(Flat trend → background RI noise is NOT the main cause of diluted ratios)",
        fontsize=10, y=1.03)
    plt.tight_layout()
    fig.savefig(os.path.join(RESULTS_DIR, "mutant_channel_enrichment_intensity.pdf"),
                bbox_inches="tight")
    plt.show()

    print(f"\nUnweighted ratio by precursor intensity quintile:")
    print(f"  {'Intensity range':<24} {'N':>6}  {'Median':>8}  {'%>1':>6}  {'%>2':>6}  {'%>5':>6}")
    print("  " + "-" * 60)
    for b in bin_order:
        grp = r_prec[r_prec["intens_bin"] == b]["ratio"].replace(0, np.nan).dropna()
        if len(grp) == 0: continue
        print(f"  10^{b.left:.1f}–10^{b.right:.1f}   {len(grp):>6,}  "
              f"{grp.median():>8.2f}  {100*(grp>1).mean():>5.1f}%  "
              f"{100*(grp>2).mean():>5.1f}%  {100*(grp>5).mean():>5.1f}%")